In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys
import pymedphys._mocks.profiles
import pymedphys._gamma.implementation.shell
import pymedphys.labs.winstonlutz.findfield
import pymedphys.labs.winstonlutz.createaxis
import pymedphys.labs.winstonlutz.interppoints
import pymedphys.labs.winstonlutz.imginterp


In [ ]:
image_path = pymedphys.data_path('wlutz_image.png')
image_path

In [ ]:
img = imageio.imread(image_path)
assert np.shape(img) == (1024, 1024)
img = img[:, 1:-1]
assert np.shape(img) == (1024, 1022)
assert img.dtype == np.dtype('uint16')
img = 1 - img[::-1,:] / 2**16
assert img.dtype == np.dtype('float64')

In [ ]:
shape = np.shape(img)
x = np.arange(-shape[1]/2, shape[1]/2)/4
y = np.arange(-shape[0]/2, shape[0]/2)/4

In [ ]:
edge_lengths = [20, 20]

centre, rotation = pymedphys.labs.winstonlutz.findfield.find_centre_and_rotation(x, y, img, edge_lengths)
centre, rotation

In [ ]:
def draw_by_diff(dx, dy, transform):
    draw_x = np.cumsum(dx)
    draw_y = np.cumsum(dy)

    draw_x, draw_y = pymedphys.labs.winstonlutz.interppoints.apply_transform(draw_x, draw_y, transform)
    
    return draw_x, draw_y

In [ ]:
transform = matplotlib.transforms.Affine2D()
transform.rotate_deg(-rotation)
transform.translate(*centre)

In [ ]:
rotation_x_points = np.linspace(-edge_lengths[0]/2, edge_lengths[0]/2, 51)
rotation_y_points = np.linspace(-edge_lengths[1]/2, edge_lengths[1]/2, 61)

rot_xx_points, rot_yy_points = np.meshgrid(rotation_x_points, rotation_y_points)

rot_xx_points, rot_yy_points = pymedphys.labs.winstonlutz.interppoints.apply_transform(rot_xx_points, rot_yy_points, transform)

In [ ]:
rotation_points = pymedphys.labs.winstonlutz.interppoints.define_rotation_field_points(centre, [20, 20], 2, rotation)

In [ ]:
rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]

rect_crosshair_dx = [-edge_lengths[0]/2, edge_lengths[0], -edge_lengths[0], edge_lengths[0]]
rect_crosshair_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1]]

plt.figure(figsize=(10,10))
plt.pcolormesh(x, y, img)
plt.plot(*draw_by_diff(rect_dx, rect_dy, transform), 'k', lw=2)
plt.plot(*draw_by_diff(rect_crosshair_dx, rect_crosshair_dy, transform), 'k', lw=0.5)

plt.scatter(centre[0], centre[1], c='r', s=1)

plt.scatter(rotation_points[0], rotation_points[1], s=1)

plt.axis('equal')
plt.xlim([-20, 20])
plt.ylim([-20, 20])

In [ ]:
def create_points_to_check(ball_bearing_diameter):
    min_dist = 0.5
    diameters = np.arange(0, ball_bearing_diameter * 1.2, min_dist)

    def points_to_check(bb_centre):
        x = []
        y = []
        for i, diameter in enumerate(diameters):
            new_x, new_y = pymedphys._gamma.implementation.shell.calculate_coordinates_shell_2d(diameter, min_dist)
            x.append(new_x + bb_centre[0])
            y.append(new_y + bb_centre[1])
        
        x = np.concatenate(x)
        y = np.concatenate(y)
        
        return x, y
    
    return points_to_check


points_to_check = create_points_to_check(8)
# points_to_check([1.46, -1.9])
plt.scatter(*points_to_check([1.46, -1.9]), s=3)
plt.axis('equal')

In [ ]:
def create_bb_to_minimise(field, ball_bearing_diameter):
    
    points_to_check = create_points_to_check(ball_bearing_diameter)
    
    def to_minimise(centre):
        

In [ ]:
plt.scatter(*pymedphys._gamma.implementation.shell.calculate_coordinates_shell_2d(3, 0.1), s=3)